# Generate Reports

In [1]:
import pandas as pd
import asyncio
import pathlib
import json
import traceback
import stamina


from kruppe.llm import OpenAILLM 

In [2]:
llm = OpenAILLM(model="gpt-4.1-mini") # using a single llm client
experiment_dir = pathlib.Path("/Users/danielliu/Workspace/fin-rag/experiments")

In [3]:
df = pd.read_csv("./reports.csv", index_col=False)
df

,category,human_report_loc,question
0,Energy (Oil),/Users/danielliu/Workspace/fin-rag/experiments...,How is ConocoPhillips positioned to deliver su...
1,Energy (Oil),/Users/danielliu/Workspace/fin-rag/experiments...,What is the outlook for Chevron Corporation’s ...
2,Energy (Oil),/Users/danielliu/Workspace/fin-rag/experiments...,What are the updated financial prospects and i...
3,Energy (Oil),/Users/danielliu/Workspace/fin-rag/experiments...,What is the investment outlook for Exxon Mobil...
4,Energy (Oil),/Users/danielliu/Workspace/fin-rag/experiments...,What is the investment outlook for ConocoPhill...
...,...,...,...
68,NVDA,/Users/danielliu/Workspace/fin-rag/experiments...,"What are the key expectations, product announc..."
69,NVDA,/Users/danielliu/Workspace/fin-rag/experiments...,"What are the current trends, risks, and invest..."
70,NVDA,/Users/danielliu/Workspace/fin-rag/experiments...,What is the current and projected financial an...
71,NVDA,/Users/danielliu/Workspace/fin-rag/experiments...,"What are the current trends, challenges, and o..."


## Kruppe

### Initialization

In [4]:
# logging imports
import logging
from logging import StreamHandler

# toolkit import
from kruppe.llm import OpenAIEmbeddingModel
from kruppe.functional.docstore.mongo_store import MongoDBStore
from kruppe.functional.rag.vectorstore.chroma import ChromaVectorStore
from kruppe.functional.rag.index.vectorstore_index import VectorStoreIndex
from kruppe.functional.rag.retriever.simple_retriever import SimpleRetriever
from kruppe.functional.rag.retriever.fusion_retriever import QueryFusionRetriever
from kruppe.functional.ragquery import RagQuery
from kruppe.functional.llmquery import LLMQuery
from kruppe.functional.newshub import NewsHub
from kruppe.functional.finhub import FinHub
from kruppe.data_source.news.nyt import NewYorkTimesData
from kruppe.data_source.news.ft import FinancialTimesData
from kruppe.data_source.news.newsapi import NewsAPIData
from kruppe.data_source.finance.yfin import YFinanceData

# researcher import
from kruppe.algorithm.librarian import Librarian
from kruppe.algorithm.coordinator import Coordinator

In [5]:
# set up logging

# handlers
formatter = logging.Formatter('%(asctime)s - %(pathname)s - %(levelname)s - %(message)s')
short_formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# set up logging for jupyter notebook
ch = StreamHandler()
ch.setFormatter(formatter)
ch.setLevel(logging.INFO)

# set up logging for everything
log_file_path = "/Users/danielliu/Workspace/fin-rag/logs/everything.log"
with open(log_file_path, 'w') as f:
    pass

fh_all = logging.FileHandler(log_file_path)
fh_all.setFormatter(formatter)
fh_all.setLevel(logging.DEBUG)

log_file_path = "/Users/danielliu/Workspace/fin-rag/logs/kruppe.log"
with open(log_file_path, 'w') as f:
    pass
fh_kruppe = logging.FileHandler(log_file_path)
fh_kruppe.setFormatter(short_formatter)
fh_kruppe.setLevel(logging.DEBUG)

# set up loggers
root_logger = logging.getLogger()
root_logger.setLevel(logging.DEBUG)
root_logger.handlers.clear()
root_logger.addHandler(fh_all) # log everything to a file

kruppe_logger = logging.getLogger("kruppe")
kruppe_logger.setLevel(logging.INFO)
kruppe_logger.handlers.clear()
kruppe_logger.addHandler(fh_kruppe) # log everything to a file
kruppe_logger.propagate = False

algo_logger = logging.getLogger("kruppe.algorithm")
algo_logger.setLevel(logging.INFO)
algo_logger.addHandler(ch) # log to console


In [6]:
reset_db=False

# db_name = "kruppe_librarian"
# collection_name = "general_news_04_20_2025"
db_name = "kruppe_librarian_2"
collection_name = "general_news_04_30_2025"

# Create doc store
unique_indices = [['title', 'datasource']] # NOTE: this is important to avoid duplicates
docstore = await MongoDBStore.acreate_db(
    db_name=db_name,
    collection_name=collection_name,
    unique_indices=unique_indices,
    reset_db=reset_db
)

# Create vectorstore index
embedding_model = OpenAIEmbeddingModel()
vectorstore = ChromaVectorStore(
    embedding_model=embedding_model,
    collection_name=collection_name,
    persist_path=f'/Volumes/Lexar/Daniel Liu/vectorstores/{db_name}'
)

index = VectorStoreIndex(vectorstore=vectorstore)
simple_retriever = SimpleRetriever(index=index)
retriever = QueryFusionRetriever(
    retrievers=[simple_retriever],
    mode='rrf',
    llm=llm,
    num_queries=3
)


if reset_db:
    vectorstore.clear()


# docs = await docstore.aget_all_documents()
# print(len(docs))
# await index.async_add_documents(docs)

print("Number of documents:", docstore.size())
print("Number of chunks:", vectorstore.size())


Number of documents: 1190
Number of chunks: 9921


In [7]:
rag_query_engine = RagQuery(
    retriever = retriever,
    llm = llm
)

llm_query_engine = LLMQuery(
    llm = llm
)

news_hub = NewsHub(news_sources=[
    NewYorkTimesData(headers_path="/Users/danielliu/Workspace/fin-rag/.nyt-headers.json"),
    FinancialTimesData(headers_path="/Users/danielliu/Workspace/fin-rag/.ft-headers.json"),
    NewsAPIData()
])

fin_hub = FinHub(
    fin_source = YFinanceData(),
    llm = llm
)

In [8]:
toolkit_librarian = [
    rag_query_engine.rag_query,
    llm_query_engine.llm_query,
    news_hub.news_search,
    news_hub.news_recent,
    news_hub.news_archive,
    fin_hub.get_company_background,
    fin_hub.get_company_income_stmt,
    fin_hub.get_company_balance_sheet,
    fin_hub.analyze_company_financial_stmts
]

toolkit_researcher = [
    rag_query_engine.rag_query,
    llm_query_engine.llm_query,
    news_hub.news_search,
    # news_hub.news_recent,
    # news_hub.news_archive,
    fin_hub.get_company_background,
    fin_hub.get_company_income_stmt,
    fin_hub.get_company_balance_sheet,
    fin_hub.analyze_company_financial_stmts
]

In [9]:
librarian = Librarian(
    llm=llm,
    toolkit=toolkit_librarian,
    docstore=docstore,
    index=index,
    max_steps=20,
    verbose=False
)

In [10]:
tree_configs = {
    "llm": llm,
    "toolkit": toolkit_researcher,
    "docstore": docstore,
    "index": index,
    "max_step": 15,
    "max_degree": 2,
    "verbose": False
}

In [11]:
n_experts = 3

### Execution

In [12]:
# coordinator = Coordinator(
#             llm=llm,
#             tree_configs = tree_configs,
#             librarian = librarian,
#         )
# reports = await coordinator.execute("How is ConocoPhillips positioned to deliver sustained free cash flow growth and shareholder returns in the coming years amid its current investment cycle and market conditions?",
#                           n_experts=n_experts)


In [13]:
# kruppe_dir = experiment_dir / "kruppe_report"
# kruppe_report_path = kruppe_dir / f"report_{0}.json"
# if coordinator.research_reports:
#     # report_dicts = coordinator.reports_to_dict()
#     research_report_dicts = [
#             report.model_dump(mode='json') for report in coordinator.research_reports
#         ]

#     report_dicts = {
#             "research_reports": research_report_dicts,
#             "background_report": coordinator._background_report.model_dump(mode='json') if coordinator._background_report else None,
#         }

#     with open(kruppe_report_path, "w") as f:
#         json.dump(report_dicts, f, indent=4)
    
#     with open(kruppe_dir / f"report_{0}_forest.html", "w") as f:
#         f.write(coordinator.visualize_research_forest())
    
#     print(f"LLM current cost: {llm.price()}")

In [14]:
import chromadb
import chromadb.errors

failed = False

kruppe_dir = experiment_dir / "kruppe_report"
kruppe_dir.mkdir(exist_ok=True)

semaphore = asyncio.Semaphore(1)  # Limit to n concurrent tasks

async def execute_with_semaphore(question: str, i: int, semaphore: asyncio.Semaphore):
    global failed
    if failed:
        print(f"Question {question} has failed. Skipping.")
        return False
    
    kruppe_report_path = kruppe_dir / f"report_{i}.json"

    if (kruppe_report_path).exists():
        # print(f"Report {i} already exists for question: {question}. Skipping.")
        return False
    
    if llm.price() > 200:
        print("LLM has exceeded the budget. Ending the experiment.")
        return False
    
    # 3 retries
    async with semaphore:
        coordinator = Coordinator(
            llm=llm,
            tree_configs = tree_configs,
            librarian = librarian,
        )

        try:
            
            for attempt in range(3):
                print(f"Report {i} - Attempt {attempt} to answer question: {question}")
                try:
                    reports = await coordinator.execute(query=question, n_experts=n_experts)
                    break  # Break if successful
                except* chromadb.errors.InternalError as e:
                    
                    if attempt == 2:
                        raise # re-raise the last exception
                    
                    # TODO
                    failed = True
                    print(f"Attempt {attempt + 1} failed with Chroma DB InternalError: {e}")
                    raise
                    # print("Resetting vectorstore and retrying...")
                    # reset_vectorstore()
                    
            
        except* Exception as eg:  # catch *all* sub-exceptions
            print(f"Error when generating report for question: {question}")
            print("Caught ExceptionGroup:")
            for sub in eg.exceptions:
                print("--- sub-exception:")
                traceback.print_exception(type(sub), sub, sub.__traceback__)

            if coordinator.research_reports:
                print("Partial report generated. Saving it.")
            else:
                print("No report generated. Skipping.")
    
    if coordinator.research_reports:
        report_dicts = coordinator.reports_to_dict()

        with open(kruppe_report_path, "w") as f:
            json.dump(report_dicts, f, indent=4)
        
        with open(kruppe_dir / f"report_{i}_forest.html", "w") as f:
            f.write(coordinator.visualize_research_forest())
        
        print(f"Report saved for question: {question} at {kruppe_report_path}")
        print(f"LLM current cost: {llm.price()}")

        return True
    
    return False

async with asyncio.TaskGroup() as tg:
    tasks = []
    for i, row in df.iterrows():
        question = row["question"]
        task = tg.create_task(execute_with_semaphore(question, i, semaphore))
        tasks.append(task)
    
results = [task.result() for task in tasks]

# Identify rows where the report was not generated successfully
failed_indices = [i for i, success in enumerate(results) if not success]
print("Rows that did not generate a report:", failed_indices)

# Optionally, inspect the corresponding rows from the DataFrame
df.loc[failed_indices]

Report 64 - Attempt 0 to answer question: Are current market estimates for U.S. software companies sufficiently conservative to account for potential first quarter macroeconomic weakness, and what are the key risks and opportunities facing major software vendors in this environment?
Background report generated for query: Are current market estimates for U.S. software companies sufficiently conservative to account for potential first quarter macroeconomic weakness, and what are the key risks and opportunities facing major software vendors in this environment?
Domain experts generated: 3 experts found from 9 generated.


2025-04-30 21:27:34,197 - /Users/danielliu/Workspace/fin-rag/src/kruppe/algorithm/hypothesis.py - ERROR - LLM did not return a valid response for the reasoning step. Missing fields: hypothesis, research_direction
2025-04-30 21:28:30,246 - /Users/danielliu/Workspace/fin-rag/src/kruppe/algorithm/hypothesis.py - ERROR - LLM did not return a valid response for the reasoning step. Missing fields: hypothesis, research_direction


Researcher eea38016-f452-4d1b-9383-11ee1ffdf87d completed with 2 reports.
Researcher 4e79b1e4-e9f0-4d03-9d03-4d06f4fd1045 completed with 2 reports.
Researcher 3f406ed7-0fd8-4138-8097-99960d278849 completed with 2 reports.
Completed 6 research reports for query: Are current market estimates for U.S. software companies sufficiently conservative to account for potential first quarter macroeconomic weakness, and what are the key risks and opportunities facing major software vendors in this environment?.
Report saved for question: Are current market estimates for U.S. software companies sufficiently conservative to account for potential first quarter macroeconomic weakness, and what are the key risks and opportunities facing major software vendors in this environment? at /Users/danielliu/Workspace/fin-rag/experiments/kruppe_report/report_64.json
LLM current cost: 0.848188
Report 70 - Attempt 0 to answer question: What is the current and projected financial and strategic outlook for NVIDIA C

2025-04-30 21:47:11,086 - /Users/danielliu/Workspace/fin-rag/src/kruppe/algorithm/hypothesis.py - ERROR - LLM did not return a valid response for the reasoning step. Missing fields: hypothesis, research_direction
2025-04-30 21:47:20,137 - /Users/danielliu/Workspace/fin-rag/src/kruppe/algorithm/hypothesis.py - ERROR - LLM did not return a valid response for the reasoning step. Missing fields: hypothesis, research_direction
2025-04-30 21:48:19,812 - /Users/danielliu/Workspace/fin-rag/src/kruppe/algorithm/hypothesis.py - ERROR - LLM did not return a valid response for the reasoning step. Missing fields: hypothesis


Researcher 12d2d535-ff81-4262-a3eb-6bbb78aed0a7 completed with 2 reports.
Researcher 8cfcfea6-6191-4fe2-8b69-f93509a633cb completed with 2 reports.
Researcher 0b0eb810-3ead-40df-a92d-340724e14777 completed with 2 reports.
Completed 6 research reports for query: What are the current trends, challenges, and outlook for valuations and fundamentals in the AI semiconductor, hardware, and networking sectors?.
Report saved for question: What are the current trends, challenges, and outlook for valuations and fundamentals in the AI semiconductor, hardware, and networking sectors? at /Users/danielliu/Workspace/fin-rag/experiments/kruppe_report/report_71.json
LLM current cost: 2.4881168000000002
Rows that did not generate a report: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 65, 66, 67, 68, 69, 72

,category,human_report_loc,question
0,Energy (Oil),/Users/danielliu/Workspace/fin-rag/experiments...,How is ConocoPhillips positioned to deliver su...
1,Energy (Oil),/Users/danielliu/Workspace/fin-rag/experiments...,What is the outlook for Chevron Corporation’s ...
2,Energy (Oil),/Users/danielliu/Workspace/fin-rag/experiments...,What are the updated financial prospects and i...
3,Energy (Oil),/Users/danielliu/Workspace/fin-rag/experiments...,What is the investment outlook for Exxon Mobil...
4,Energy (Oil),/Users/danielliu/Workspace/fin-rag/experiments...,What is the investment outlook for ConocoPhill...
...,...,...,...
66,IT-Application Software,/Users/danielliu/Workspace/fin-rag/experiments...,What is the current financial performance and ...
67,IT-Application Software,/Users/danielliu/Workspace/fin-rag/experiments...,Is Adobe Inc. positioned for sustained growth ...
68,NVDA,/Users/danielliu/Workspace/fin-rag/experiments...,"What are the key expectations, product announc..."
69,NVDA,/Users/danielliu/Workspace/fin-rag/experiments...,"What are the current trends, risks, and invest..."
